In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy.io import wavfile
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import os

# --- Filter Functions ---

def butter_lowpass_filter(data, cutoff, fs, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data)

def butter_highpass_filter(data, cutoff, fs, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return filtfilt(b, a, data)

def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band', analog=False)
    return filtfilt(b, a, data)

def butter_bandstop_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='bandstop', analog=False)
    return filtfilt(b, a, data)

# --- GUI Logic ---

def upload_file():
    global audio_data, sample_rate, time
    file_path = filedialog.askopenfilename(filetypes=[("WAV Files", "*.wav")])
    if file_path:
        try:
            sample_rate, audio_data = wavfile.read(file_path)
            if audio_data.ndim > 1:  # Handle stereo audio by averaging channels
                audio_data = audio_data.mean(axis=1)
            
            # Create time axis
            time = np.linspace(0, len(audio_data) / sample_rate, num=len(audio_data))
            
            messagebox.showinfo("Success", "Audio file loaded successfully!")
            plot_original_audio()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load audio: {e}")

def plot_original_audio():
    fig.clear()
    ax1 = fig.add_subplot(211)
    ax1.plot(time, audio_data)
    ax1.set_title("Original Audio Signal")
    ax1.set_xlabel("Time (s)")
    ax1.set_ylabel("Amplitude")
    canvas.draw()

def apply_filter():
    global filtered_data
    if audio_data is None:
        messagebox.showerror("Error", "Please upload an audio file first!")
        return

    filter_type = filter_type_var.get()
    
    try:
        if filter_type == "Low-pass":
            cutoff = cutoff_var.get()
            filtered_data = butter_lowpass_filter(audio_data, cutoff, sample_rate)
        elif filter_type == "High-pass":
            cutoff = cutoff_var.get()
            filtered_data = butter_highpass_filter(audio_data, cutoff, sample_rate)
        elif filter_type == "Band-pass":
            lowcut = lowcut_var.get()
            highcut = highcut_var.get()
            filtered_data = butter_bandpass_filter(audio_data, lowcut, highcut, sample_rate)
        elif filter_type == "Band-stop":
            lowcut = lowcut_var.get()
            highcut = highcut_var.get()
            filtered_data = butter_bandstop_filter(audio_data, lowcut, highcut, sample_rate)
        
        plot_filtered_audio()
        
    except Exception as e:
        messagebox.showerror("Error", f"Error applying filter: {e}")

def plot_filtered_audio():
    fig.clear()
    
    # Re-plot original to keep context
    ax1 = fig.add_subplot(211)
    ax1.plot(time, audio_data)
    ax1.set_title("Original Audio Signal")
    ax1.set_xlabel("Time (s)")
    ax1.set_ylabel("Amplitude")
    
    # Plot filtered
    ax2 = fig.add_subplot(212)
    ax2.plot(time, filtered_data)
    ax2.set_title("Filtered Audio Signal")
    ax2.set_xlabel("Time (s)")
    ax2.set_ylabel("Amplitude")
    
    fig.tight_layout() # Adjust layout to prevent overlap
    canvas.draw()

def save_filtered_audio():
    if filtered_data is None:
        messagebox.showerror("Error", "No filtered audio to save!")
        return
    
    file_path = filedialog.asksaveasfilename(defaultextension=".wav", filetypes=[("WAV Files", "*.wav")])
    if file_path:
        try:
            # Convert back to int16 for standard WAV format
            wavfile.write(file_path, sample_rate, filtered_data.astype(np.int16))
            messagebox.showinfo("Success", "Filtered audio saved successfully!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save audio: {e}")

# --- Initialize GUI ---

root = tk.Tk()
root.title("Audio Signal Filtering")
root.geometry("800x800")

# Global variables
audio_data = None
filtered_data = None
sample_rate = None
time = None

# Variables for inputs
filter_type_var = tk.StringVar(value="Low-pass")
cutoff_var = tk.DoubleVar(value=1000)
lowcut_var = tk.DoubleVar(value=300)
highcut_var = tk.DoubleVar(value=3000)

# Control Frame
frame = tk.Frame(root)
frame.pack(pady=10)

# Grid Layout for Controls
tk.Button(frame, text="Upload Audio", command=upload_file).grid(row=0, column=0, padx=5, pady=5)

tk.Label(frame, text="Filter Type:").grid(row=1, column=0, sticky="e")
filter_menu = ttk.Combobox(frame, textvariable=filter_type_var, values=["Low-pass", "High-pass", "Band-pass", "Band-stop"])
filter_menu.grid(row=1, column=1, padx=5, pady=5)

tk.Label(frame, text="Cutoff Frequency:").grid(row=2, column=0, sticky="e")
tk.Entry(frame, textvariable=cutoff_var).grid(row=2, column=1, padx=5, pady=5)

tk.Label(frame, text="Low Cutoff:").grid(row=3, column=0, sticky="e")
tk.Entry(frame, textvariable=lowcut_var).grid(row=3, column=1, padx=5, pady=5)

tk.Label(frame, text="High Cutoff:").grid(row=4, column=0, sticky="e")
tk.Entry(frame, textvariable=highcut_var).grid(row=4, column=1, padx=5, pady=5)

tk.Button(frame, text="Apply Filter", command=apply_filter).grid(row=5, column=0, pady=10)
tk.Button(frame, text="Save Filtered Audio", command=save_filtered_audio).grid(row=5, column=1, pady=10)

# Plotting Area
fig = plt.Figure(figsize=(8, 6))
canvas = FigureCanvasTkAgg(fig, root)
canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

root.mainloop()